In [1]:
from sqlanalyzer import column_parser
import pandas as pd
import sqlparse
import re

In [2]:
from sqlanalyzer import column_parser
import pandas as pd
import sqlparse
import re


def delevel_query(query_list):

    pos_delete = [len(query_list)-1]
    pos_join = []
    for i, line in enumerate(query_list):
        if line.startswith('ORDER') or line.startswith('GROUP'):
            pos_delete.append(i)
        if line.startswith('FROM') and len(line.split(' ')) > 1:
            pos_join.append(i)
        elif line.startswith('FROM') and len(line.split(' ')) == 1:
            pos_join.append(i+1)
        if line.startswith('WHERE'):
            pos_where = i
        if line.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN'):
            pos_join.append(i+1)

    pos_join.append(min(pos_delete))

    sub_query = {}
    pos_join_list = iter(pos_join)
    next(pos_join_list)
    alias_pos = []

    if query_list[pos_join[0]].startswith('FROM'):
        alias_pos.append(pos_join[0])

    for i in range(len(pos_join)-1):
        if i < len(pos_join)-2 and pos_join[i] < pos_where:
            end_pos = next(pos_join_list)-1
            alias_pos.append(end_pos-1)

        elif pos_join[-1] >= pos_where:
            end_pos = next(pos_join_list)-1
            alias_pos.append(pos_where - 1)

        else:
            end_pos = pos_join[-1]

    alias_pos = list(set(alias_pos))
    
    sub_query = {}

    for j in alias_pos:
        alias = query_list[j]
        try:
            alias_index = alias.split(' ')[::-1].index('ON')
            alias = alias.split(' ')[::-1][alias_index+1]
        except:
            alias = alias.split(' ')[-1]

        sub_query[alias] = ' '.join(query_list[pos_join[i] : end_pos])
            
    return sub_query


# def has_child(sub_query_list):
#     query_list = parse_sub_query(sub_query_list)
#     query_list = delevel_query(query_list)
#     return query_list, query_list != {}


In [3]:
query = """SELECT sfdc_accounts.platform, sfdc_accounts.mobile_os, sfdc_accounts.service_metadata,
sfdc_cases.account, sfdc_cases.num_requests, sfdc_cases.owner
FROM sfdc.accounts sfdc_accounts
LEFT JOIN (SELECT MAX(dt) FROM (SELECT dt FROM sfdc.oppty) sfdc_oppty LEFT JOIN (SELECT dt FROM sfdc.cases) sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt) 
AS sfdc_cases_oppty ON sfdc_cases_oppty.dt = sfdc_accounts.dt
WHERE sfdc_cases_oppty.dt > '2020-04-03' AND sfdc_cases_oppty.dt < '2020-05-04' ORDER BY 1 GROUP BY 3 LIMIT 20
"""


## delevel_query

## level 0 -- level 1 -- level 2

In [4]:
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
query_list_0 = formatted_query.split('\n')

In [5]:
sub_query = delevel_query(query_list_0)
sub_query


{'sfdc_accounts': "  (SELECT MAX(dt)    FROM      (SELECT dt       FROM sfdc.oppty) sfdc_oppty    LEFT JOIN      (SELECT dt       FROM sfdc.cases) sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt) AS sfdc_cases_oppty ON sfdc_cases_oppty.dt = sfdc_accounts.dt WHERE sfdc_cases_oppty.dt > '2020-04-03'",
 'sfdc_cases_oppty': "  (SELECT MAX(dt)    FROM      (SELECT dt       FROM sfdc.oppty) sfdc_oppty    LEFT JOIN      (SELECT dt       FROM sfdc.cases) sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt) AS sfdc_cases_oppty ON sfdc_cases_oppty.dt = sfdc_accounts.dt WHERE sfdc_cases_oppty.dt > '2020-04-03'"}

In [6]:
sub_query.keys()

dict_keys(['sfdc_accounts', 'sfdc_cases_oppty'])

In [7]:
for query_alias in sub_query.keys():
    query = sub_query[query_alias].lstrip('\n \(')
    print(query_alias, query)
    formatter = column_parser.Parser(query)
    formatted_sub_query = formatter.format_query(query)
    sub_sub_query = delevel_query(formatted_sub_query.split('\n'))
#     print(query_alias, query)


sfdc_accounts SELECT MAX(dt)    FROM      (SELECT dt       FROM sfdc.oppty) sfdc_oppty    LEFT JOIN      (SELECT dt       FROM sfdc.cases) sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt) AS sfdc_cases_oppty ON sfdc_cases_oppty.dt = sfdc_accounts.dt WHERE sfdc_cases_oppty.dt > '2020-04-03'
sfdc_cases_oppty SELECT MAX(dt)    FROM      (SELECT dt       FROM sfdc.oppty) sfdc_oppty    LEFT JOIN      (SELECT dt       FROM sfdc.cases) sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt) AS sfdc_cases_oppty ON sfdc_cases_oppty.dt = sfdc_accounts.dt WHERE sfdc_cases_oppty.dt > '2020-04-03'


## refactor

In [8]:
query_list = query_list_0

In [9]:
query_list

['SELECT sfdc_accounts.platform,',
 '       sfdc_accounts.mobile_os,',
 '       sfdc_accounts.service_metadata,',
 '       sfdc_cases.account,',
 '       sfdc_cases.num_requests,',
 '       sfdc_cases.owner',
 'FROM sfdc.accounts sfdc_accounts',
 'LEFT JOIN',
 '  (SELECT MAX(dt)',
 '   FROM',
 '     (SELECT dt',
 '      FROM sfdc.oppty) sfdc_oppty',
 '   LEFT JOIN',
 '     (SELECT dt',
 '      FROM sfdc.cases) sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt) AS sfdc_cases_oppty ON sfdc_cases_oppty.dt = sfdc_accounts.dt',
 "WHERE sfdc_cases_oppty.dt > '2020-04-03'",
 "  AND sfdc_cases_oppty.dt < '2020-05-04'",
 'ORDER BY 1',
 'GROUP BY 3',
 'LIMIT 20']

In [10]:
sub_query = []
def get_joins_pos(query_list):

    pos_delete = [len(query_list)-1]
    pos_join = []
    for i, line in enumerate(query_list):
        if line.startswith('ORDER') or line.startswith('GROUP'):
            pos_delete.append(i)
        if line.startswith('FROM') and len(line.split(' ')) > 1:
            pos_join.append(i)
        elif line.startswith('FROM') and len(line.split(' ')) == 1:
            pos_join.append(i+1)
        if line.startswith('WHERE'):
            pos_where = i
        if line.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN'):
            pos_join.append(i+1)

    pos_join.append(min(pos_delete))
    return pos_join, pos_where


In [11]:
pos_join, pos_where = get_joins_pos(query_list)
pos_join, pos_where

([6, 8, 17], 15)

In [12]:
sub_query = {}

def get_alias_pos(query_list, pos_join, pos_where):
    pos_join_list = iter(pos_join)
    next(pos_join_list)
    alias_pos = []

    if query_list[pos_join[0]].startswith('FROM'):
        alias_pos.append(pos_join[0])

    for i in range(len(pos_join)-1):
        if i < len(pos_join)-2 and pos_join[i] < pos_where:
            end_pos = next(pos_join_list)-1
            alias_pos.append(end_pos-1)

        elif pos_join[-1] >= pos_where:
            end_pos = next(pos_join_list)-1
            alias_pos.append(pos_where - 1)

        else:
            end_pos = pos_join[-1]

    alias_pos = list(set(alias_pos))
    return alias_pos


In [13]:
alias_pos = get_alias_pos(query_list, pos_join, pos_where)
alias_pos

[6, 14]

In [14]:
for i, line in enumerate(query_list): print(i, line)

0 SELECT sfdc_accounts.platform,
1        sfdc_accounts.mobile_os,
2        sfdc_accounts.service_metadata,
3        sfdc_cases.account,
4        sfdc_cases.num_requests,
5        sfdc_cases.owner
6 FROM sfdc.accounts sfdc_accounts
7 LEFT JOIN
8   (SELECT MAX(dt)
9    FROM
10      (SELECT dt
11       FROM sfdc.oppty) sfdc_oppty
12    LEFT JOIN
13      (SELECT dt
14       FROM sfdc.cases) sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt) AS sfdc_cases_oppty ON sfdc_cases_oppty.dt = sfdc_accounts.dt
15 WHERE sfdc_cases_oppty.dt > '2020-04-03'
16   AND sfdc_cases_oppty.dt < '2020-05-04'
17 ORDER BY 1
18 GROUP BY 3
19 LIMIT 20


In [15]:
sub_query_pos = list(zip(pos_join[:-1], alias_pos))
sub_query_pos

[(6, 6), (8, 14)]

In [16]:
def parse_sub_query(query_list, sub_query_pos):
    sub_query = {}
    for sub_index, sub_pos in enumerate(sub_query_pos):
        alias = query_list[sub_pos[1]]
        query = query_list[sub_pos[0]: sub_pos[1]]

        try:
            alias_list_rev = alias.split(' ')[::-1]
            alias_index = alias_list_rev.index('ON')
            alias = alias_list_rev[alias_index+1]

            if alias_list_rev[alias_index+2] == 'AS':
                del alias_list_rev[:alias_index+3]

            else:
                del alias_list_rev[:alias_index+2]

            query.append(' '.join(alias_list_rev[::-1]).rstrip('\)').lstrip(' '))

        except:
            query.append(' '.join(alias.split(' ')[:-1]).rstrip('\)').lstrip(' '))
            alias = alias.split(' ')[-1]

        sub_query[alias] = ' '.join(query).lstrip(' \(')
        
    return sub_query


In [17]:
sub_query = parse_sub_query(query_list, sub_query_pos)
sub_query


{'sfdc_accounts': 'FROM sfdc.accounts',
 'sfdc_cases_oppty': 'SELECT MAX(dt)    FROM      (SELECT dt       FROM sfdc.oppty) sfdc_oppty    LEFT JOIN      (SELECT dt FROM sfdc.cases) sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt'}